# insert titel of code

generic intro text

## imports

### Libraries

In [3]:
import pickle

### Custom scripts

In [1]:
from scripts.args import *
from scripts.base_model.preprocessing import *

## Basemodel

### Preprocessing

In [2]:
# get the arguments for preprocessing
preprocessing_args, unkown = arg_parser_preprocessing()

# load dataset if the dataset exist
dataset_path = preprocessing_args.save_path
if preprocessing_args.use_pre:
    with open(dataset_path, "rb") as f:
        dataset = pickle.load(f)
else:
    dataset = preprocessing(preprocessing_args)
    with open(dataset_path, "wb") as f:
            pickle.dump(dataset, f)

======================= sample training data =======================
# training samples : 285879
======================= sample test data =======================
# test samples : 5105


### Train base model